## **Rabin's Cryptosystem**

Rabin's Cryptosystem is a public-key cryptosystem, meaning it uses a pair of keys: a public key for encryption and a private key for decryption. It was introduced by Michael O. Rabin in 1979 and holds historical significance as one of the earliest provably secure public-key cryptosystems. Key Features:

1.  **Public-Key Nature:** It utilizes distinct public and private keys.
2.  **Provable Security:** Rabin's Cryptosystem is one of the few cryptosystems whose security can be mathematically proven to be equivalent to the difficulty of factoring large integers.
3.  **Ambiguity of Decryption:** A unique characteristic of Rabin's Cryptosystem is that decryption yields four possible plaintexts for a given ciphertext.[texto do link](https://)
4.  **Efficiency:** Encryption is very efficient, involving only squaring modulo N. Decryption is also relatively efficient, but involves modular square roots.

### **Rabin's Cryptosystem - Mathematical Foundations**
Rabin's Cryptosystem is an asymmetric encryption technique whose security is based on the computational difficulty of finding square roots modulo a large composite number, which is equivalent to factoring the modulus. It is notable for being the first public-key cryptosystem proven to be as secure as factoring.

### 1. Key Generation
In Rabin's Cryptosystem, the key generation process involves selecting large prime numbers and computing a modulus.

*   **Selection of Primes:** Choose two large, distinct prime numbers, $p$ and $q$. For the system to work efficiently and securely, $p$ and $q$ should both be congruent to $3 \pmod 4$. This property simplifies the calculation of square roots modulo $p$ and $q$.
    *   $p \equiv 3 \pmod 4$
    *   $q \equiv 3 \pmod 4$

*   **Modulus Calculation:** Compute the modulus $n$ as the product of the two primes:
    $$n = p \cdot q$$

*   **Public Key Parameter `b`:** A public key parameter $b$ is chosen, where $0 \le b < n$. Often, for simplicity, $b=0$ is used.

*   **Public Key:** The public key is $(n, b)$. This key is shared with anyone who wishes to encrypt a message.

*   **Private Key:** The private key is $(p, q)$. These prime factors must be kept secret for secure decryption.

### 2. Encryption
To encrypt a plaintext message $m$, where $0 \le m < n$, the sender uses the recipient's public key $(n, b)$. The encryption process squares the message and adds $b \cdot m$ modulo $n$.

*   **Encryption Formula:** The ciphertext $c$ is computed as:
    $$c = m(m+b) \pmod n$$
    This can also be written as:
    $$c = (m^2 + bm) \pmod n$$

### 3. Decryption
Decryption is more complex as it involves finding the inverse of the encryption function, which is equivalent to finding the square root of $c$ modulo $n$. This typically yields four possible plaintexts.

*   **Goal:** Solve the quadratic congruence $m^2 + bm - c \equiv 0 \pmod n$ for $m$. This can be rewritten as $(m + b/2)^2 \equiv c + (b/2)^2 \pmod n$. To avoid issues with $b/2$, an alternative approach is often used directly with the definition of Rabin's Cryptosystem.

    We want to find $m$ such that $m^2 + bm \equiv c \pmod n$.

*   **Decryption Steps (for $b=0$, simplifies to $m^2 \equiv c \pmod n$):**

    1.  **Solve modulo $p$ and $q$:** Find the square roots of $c$ modulo $p$ and modulo $q$ separately.
        *   Find $m_p \equiv \pm c^{(p+1)/4} \pmod p$. (This works because $p \equiv 3 \pmod 4$, so $(p+1)/4$ is an integer).
        *   Find $m_q \equiv \pm c^{(q+1)/4} \pmod q$. (Similarly, this works because $q \equiv 3 \pmod 4$).
        This gives two solutions modulo $p$ ($m_p$, $-m_p$) and two solutions modulo $q$ ($m_q$, $-m_q$).

    2.  **Combine using Chinese Remainder Theorem (CRT):** Use the CRT to combine these four pairs of solutions to find the four possible values for $m$ modulo $n$. For each combination $(x_p, x_q)$ where $x_p \in \{m_p, -m_p\}$ and $x_q \in \{m_q, -m_q\}$, solve the system of congruences:
        $$\begin{cases} x \equiv x_p \pmod p \\ x \equiv x_q \pmod q \end{cases}$$
        The CRT guarantees a unique solution modulo $n$ for each pair. For instance, to find $x$, we can use the formula:
        $$x \equiv x_p \cdot q \cdot (q^{-1} \pmod p) + x_q \cdot p \cdot (p^{-1} \pmod q) \pmod n$$

        This process yields four potential plaintext messages. The recipient must then determine which of these four is the original message, often by checking for known message formats or redundancy.

In [1]:
#Import Library
import random

In [2]:
#Create Functions
def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    else:
        g, x, y = extended_gcd(b % a, a)
        return g, y - (b // a) * x, x

def modular_inverse(a, m):
    g, x, y = extended_gcd(a, m)
    if g != 1:
        raise ValueError("The modular inverse does not exist (a and m are not coprime).")
    else:
        return x % m

In [3]:
#Create Power and Prime Functions
def power(a, b, m):
    """Calculates (a^b) % m using modular exponentiation."""
    res = 1
    a %= m
    while b > 0:
        if b % 2 == 1:
            res = (res * a) % m
        a = (a * a) % m
        b //= 2
    return res

def is_prime(n, k=5):
    """Miller-Rabin primality test to check if n is likely prime.
    k is the number of iterations (higher k means higher certainty).
    """
    if n < 2:
        return False
    if n == 2 or n == 3:
        return True
    if n % 2 == 0:
        return False

    # Write n-1 as 2^s * d
    s = 0
    d = n - 1
    while d % 2 == 0:
        d //= 2
        s += 1

    # Witness loop
    for _ in range(k):
        a = random.randint(2, n - 2)
        x = power(a, d, n)
        if x == 1 or x == n - 1:
            continue
        composite = True
        for _ in range(s - 1):
            x = power(x, 2, n)
            if x == n - 1:
                composite = False
                break
        if composite:
            return False # n is composite
    return True # n is probably prime

In [4]:
#Function to Create a a random number of 'bits' length
def generate_prime(bits):
    while True:
        num = random.getrandbits(bits)
        num |= (1 << bits - 1)
        num |= 1

        if is_prime(num):
            if num % 4 == 3:
                return num

In [5]:
#Function to Generate Keys
def generate_keys(bits):
    while True:
        p = generate_prime(bits) # Generate first prime p
        q = generate_prime(bits) # Generate second prime q
        if p != q:
            break

    n = p * q
    b = 0

    public_key = (n, b)
    private_key = (p, q)

    return public_key, private_key

In [6]:
#Encryption Function
def encrypt(m, public_key):
    n, b = public_key

    # Check if the message is within the valid range [0, n-1]
    if not (0 <= m < n):
        raise ValueError("Plaintext message m must be in the range [0, n-1].")

    # Encryption formula: c = m * (m + b) mod n
    c = (m * (m + b)) % n
    return c

In [7]:
#Find the Square Roots in Prime Values - Function
def find_square_roots_mod_prime(c, p):
    exponent = (p + 1) // 4
    r1 = power(c, exponent, p)
    r2 = p - r1
    return r1, r2

In [8]:
#Function to Apply Chinese Remainder Theorem
def chinese_remainder_theorem(a1, n1, a2, n2):
    g, x, y = extended_gcd(n1, n2)
    if g != 1:
        raise ValueError("n1 and n2 are not coprime, CRT not applicable directly.")

    inv_n1_mod_n2 = modular_inverse(n1, n2)
    inv_n2_mod_n1 = modular_inverse(n2, n1)

    result = (a1 * n2 * inv_n2_mod_n1 + a2 * n1 * inv_n1_mod_n2) % (n1 * n2)
    return result

In [9]:
#Decrypt Function
def decrypt(c, private_key, b):
    if b != 0:
        raise NotImplementedError("Decryption for b != 0 is more complex and not implemented here.")

    p, q = private_key
    n = p * q

    r_p1, r_p2 = find_square_roots_mod_prime(c, p)
    r_q1, r_q2 = find_square_roots_mod_prime(c, q)
    plaintexts = []

    plaintexts.append(chinese_remainder_theorem(r_p1, p, r_q1, q))
    plaintexts.append(chinese_remainder_theorem(r_p1, p, r_q2, q))
    plaintexts.append(chinese_remainder_theorem(r_p2, p, r_q1, q))
    plaintexts.append(chinese_remainder_theorem(r_p2, p, r_q2, q))

    return plaintexts

In [10]:
#Application Example
#Step 01 - Key Generation

BIT_LENGTH = 16 # Use a small bit length for demonstration purposes to keep numbers manageable
print(f"\nGenerating Rabin keys with bit length: {BIT_LENGTH}")
public_key, private_key = generate_keys(BIT_LENGTH)
n, b = public_key
p, q = private_key

print(f"Public Key (n, b): ({n}, {b})")
print(f"Private Key (p, q): ({p}, {q})")
print(f"p %% 4: {p % 4}")
print(f"q %% 4: {q % 4}")


Generating Rabin keys with bit length: 16
Public Key (n, b): (1998701129, 0)
Private Key (p, q): (39667, 50387)
p %% 4: 3
q %% 4: 3


In [11]:
#Step 02 - Message Preparation ( Validation )
original_message = 12345

print(f"\nOriginal Message (m): {original_message}")
if not (0 <= original_message < n):
    print(f"Warning: Original message {original_message} is not in range [0, {n-1}]. Adjusting...")

    if original_message >= n:
        original_message = original_message % n
        if original_message == 0:
            original_message = 1
    print(f"Adjusted Message (m): {original_message}")



Original Message (m): 12345


In [12]:
#Step 3 - Encryption
ciphertext = encrypt(original_message, public_key)
print(f"Ciphertext (c): {ciphertext}")

Ciphertext (c): 152399025


In [13]:
#Step 4 - Decryption
decrypted_plaintexts = decrypt(ciphertext, private_key, b)
print(f"Four possible decrypted plaintexts: {decrypted_plaintexts}")

Four possible decrypted plaintexts: [1998688784, 1534346882, 464354247, 12345]


In [14]:
#Step 5 - Plaintext Recovery
recovered_message = None
for m_candidate in decrypted_plaintexts:
    if m_candidate == original_message:
        recovered_message = m_candidate
        break

    if b == 0 and (n - m_candidate) == original_message:
        recovered_message = n - m_candidate
        break

if recovered_message is not None:
    print(f"Original message successfully recovered: {recovered_message}")
else:
    print("Could not unambiguously recover the original message from the four candidates.")
    print(f"Original: {original_message}")
    print(f"Candidates: {decrypted_plaintexts}")

Original message successfully recovered: 12345
